<a href="https://colab.research.google.com/github/DCA-28/DCA-28/blob/main/Trabalho_1_filas_e_ru%C3%ADnas_do_apostador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabalho 1
##Filas e Ruína do Apostador
##Modelagem e Avaliação de Desempenho

## Entrega Preliminar

### Alunos:
Daniel Corcino de Albuquerque - DRE: 118188457

Letícia Freire Carvalho de Sousa - DRE: 118025324

Lucas Favilla Ferreira Alves da Silva - DRE: 119156518

Roberto Leonie Ferreira Moreira - DRE: 116062192



## Simulação

Fizemos uma simulação em que passamos como parâmetros:

$\lambda$ - taxa da variável exponencial que modela o tempo entre as chegadas dos clientes  

$\mu$ - taxa da variável exponencial que modela a vida residual dos clientes

`LIM_TIME` - Tempo limite que passamos aceitando novos clientes na fila de espera

Após passarmos do tempo limite terminamos a simulação com os clientes que já estavam na fila de espera.


Estamos computando os ciclos de vida do servidor (do nascimento a morte) bem como seus ciclos ociosos.

### Métricas

Para entrega preliminar computamos:

  $E(N)$ = valor esperado do número de clientes no sistema, a cada momento

  $E(Nq)$ = número de clientes na fila de espera, a cada chegada de novo cliente

  $E(X)$ = valor esperado do tempo de serviço prestado a cada cliente

  $E(W)$ = valor esperado do tempo na fila de espera de cada cliente

  $E(T)$ = valor esperado do tempo de cada cliente no sistema


In [ ]:
import numpy as np
import random
import copy
import time

# A simulacao em si
def simulation(LAMBDA, MU, LIM_TIME, VERBOSE = 0):

  # Medidas interessantes a serem analisadas
  class Measures:
    def __init__(self):
      #variaveis auxiliares para as medidas
      self.TotalClients = 0 # número total de clientes que passaram pelo sistema
      self.sum_Nq = 0 #  somatorio do número de clientes na fila de espera a cada vez que um cliente chega
      self.N = 0 # total medio de clientes no sistema a cada momento
      self.T = 0 # somatorio do tempo total gasto no sistema por cada cliente
      self.W = 0 # somatorio do tempo total gasto na fila de espera por cada cliente
      self.X = 0 # somatorio do tempo total gasto em serviço por cada cliente

      self.count_idle_cycles = 0 # numero de ciclos ociosos
      self.count_birth_and_death_processes = 0 # numero de ciclos de vida e morte
      self.sum_idle_cycles_time = 0 # somatorio da duração de cada ciclo ocioso
      self.sum_birth_and_death_processes_time = 0 # somatorio da duração de cada ciclo de vida e morte
      self.sum_time = 0 # tempo total de simulação

    # valor esperado do número de clientes no sistema, a cada momento
    def Expected_Value_N(self):
      return self.N / (self.sum_time)

    # valor esperado do número de clientes na fila de espera, a cada chegada de novo cliente
    def Expected_Value_Nq(self):
      return self.sum_Nq / self.TotalClients

    # valor esperado da porcentagem do tempo em que o sistema está ativo
    def Expected_Value_Rho(self):
      return self.sum_birth_and_death_processes_time / (self.sum_time)

    # valor esperado do tempo de serviço prestado a cada cliente
    def Expected_Value_X(self):
      return self.X / self.TotalClients

    # valor esperado do tempo na fila de espera de cada cliente
    def Expected_Value_W(self):
      return self.W / self.TotalClients

    # valor esperado do tempo de cada cliente no sistema
    def Expected_Value_T(self):
      return self.T / self.TotalClients

  # classe que modela os eventos de chegada e saída de cada cliente no sistema
  # id -- id do cliente -- incremental a partir de 1
  # event_type -- tipo do evento ("A" - chegada, "D" - saída)
  # arrival_time -- momento da chegada no sistema
  # departure_time -- momento de saída do sistema
  # service_start -- momento em que começa a ser serviço
  # time -- momento atual, pode ser igual a arrival_time ou departure_time, a depender
  #         de qual evento está sendo modelado

  class Event:
    def __init__(self, event_type, time, id):
      self.event_type = event_type
      self.time = time
      self.service_start = -1
      if(event_type == 'A'):
        self.arrival_time = self.time
        self.departure_time = -1
      else:
        self.departure_time = self.time
      self.id = id

    # para imprimir de forma expressiva os dados do cliente
    def __str__(self):
      if(self.event_type == "A"):
        return str(self.id) + "\t|\t"+str(f"{self.arrival_time:.3f}").format()+ "\t|"
      return str(self.id) + "\t|\t" + str(f"{self.arrival_time:.3f}") + "\t|\t"
      + str(f"{self.service_start:.3f}") + "\t|\t" + str(f"{self.departure_time:.3f}") + "\t|"

    # comparador, utilizado para sabermos que evento vem primeiro
    # em caso de empate damos prioridade para a saída do sistema
    def __lt__(self, other):
      if(self.time<other.time):
          return True
      if(self.time>other.time):
          return False
      else:
          return (self.event_type == 'D')

  # o tempo de vida residual é modelado por uma variavel aleatória exponencial com taxa MU
  def residual_life():
    return random.expovariate(MU)

  # o momento da proxima chegada é modelado pela soma de variaveis aleatórias exponenciais iid com taxa LAMBDA
  ARRIVAL_TIME = 0
  def next_arrival():
    return ARRIVAL_TIME + random.expovariate(LAMBDA)

  # classe para modelar o sistema
  # waiting_queue -- lista que representa a fila de espera (células são do tipo Event)
  # server -- variável do tipo Event que representa o cliente que está sendo servido
  class System:
    def __init__(self):
      self.waiting_queue = []
      self.server = None

    # cliente e entra na lista de espera
    def arrival(self, e):
      self.waiting_queue.append(e)

    # cliente que está sendo servido sai do sistema
    # garantidamente só é chamado se o sistema estava ocupado
    def departure(self):
      server = copy.deepcopy(self.server)
      self.server = None
      return server

    # o próximo cliente sai da fila de espera e é servido
    def next(self, T):
      if(self.busy() or not(self.waiting_queue)):
        return None
      self.server = self.waiting_queue.pop(0)
      x = residual_life()
      measures.X += x
      self.server.departure_time = T + x
      self.server.time = self.server.departure_time
      return self.server

    # verifica se o sistema está ocupado
    def busy(self):
      return not(self.server == None)

    # tamanho da fila de espera
    def waiting_queue_size(self):
      return len(self.waiting_queue)

    # tamanho da fila (fila de espera + servidor)
    def full_queue_size(self):
      s = len(self.waiting_queue)
      if(self.busy()):
        s += 1
      return s

    # para imprimir de forma expressiva os dados dos clientes na fila
    # imprimimos os dados de no máximo 5 clientes para não poluir a simulação
    def __str__(self):
      ans = "\tServidor: \n"
      ans += "\033[1m"+"\tid\t|\tchegada\t|\tservidor|\tsaída\t|\n"+"\033[0m"
      ans += "\t" + str(self.server) + "\n"


      ans += "\n \tFila de Espera: ("+ str(self.waiting_queue_size())+ ") \n"

      ans += "\033[1m"+"\tid\t|\tchegada\t|\n"+"\033[0m"

      lim = 5
      for w in self.waiting_queue:
        ans += "\t" + str(w) + " \n"
        lim -= 1
        if(lim == 0):
          ans += "\t...\n"
          break
      ans += "---------------------------------------\n"
      return ans


  # variáveis principais
  system = System()
  measures = Measures()

  id = 0 # id do próximo cliente que chegará

  ARRIVAL_TIME = 0
  # evento de chegada do próximo cliente
  # é inicializado fora do laço principal para que a simulção comece com um cliente no sistema
  next_arrival_event = Event('A', ARRIVAL_TIME, id)

  # evento de saída do cliente que está no servidor
  # None se o sistema está ocioso, ou antes da primeira iteração do laço principal
  next_departure_event = None

  T = -1 # tempo atual
  birth = -1 # tempo do último nascimento, -1 se ocioso
  death = -1 # tempo da última morte, -1 se é o primeiro ciclo de vida e morte
  count_clients = 0 # número total de clientes que passaram pelo sistema no ciclo de vida e morte atual

  # laço principal da simulação
  # Quando a simulação chega ao tempo limite paramos de aceitar a chegada de novos clientes
  # e, após, a simulação para depois de atender todos os clientes que já estavam na fila
  while(True):
    # evento atual, pode ser de chegada ou de saída
    current_event = []

    # se o evento atual é de chegada
    if(not(next_departure_event) or (not(next_arrival_event == None) and next_arrival_event.time < next_departure_event.time) ):
      # atualizamos a métrica do número de clientes na fila de espera no momento de chegada do cliente
      measures.sum_Nq += system.waiting_queue_size()

      current_event = next_arrival_event
      id += 1
      next_arrival_event = None
      # aceitamos nova chegada na sila de espera se não terminou o tempo da simulação
      if(T < LIM_TIME):
        ARRIVAL_TIME = next_arrival()
        next_arrival_event = Event('A', ARRIVAL_TIME, id)

    # se o evento atual é de partida
    else:
      current_event = next_departure_event

      # atualizamos a métrica do tempo total do cliente no sistema
      measures.T += next_departure_event.departure_time - next_departure_event.arrival_time

      next_departure_event = None


    if(VERBOSE > 1):
      if(current_event.event_type == 'A'):
        print("Evento de Chegada")
      else:
        print("Evento de Saída")
      print("\033[1m"+"id\t|\tchegada\t|\tservidor|\tsaída\t|\n"+"\033[0m")
      print(current_event)

    # atualizamos tempo atual da simulação
    last_T = T
    T = current_event.time

    if(VERBOSE > 1):
      print(f"\n Tempo do evento atual: {T:.3f}\n")

    # se o evento atual é de chegada
    if(current_event.event_type == 'A'):
      count_clients += 1
      if(birth == -1):
        birth = T

        # se estamos começando novo ciclo de vida e morte
        if(not(death == -1)):
          # atualizamos várias métricas:

          # somatório do número de clientes no sistema a cada momento
          measures.N += system.full_queue_size() * (T - last_T)

          # número de ciclos ociosos
          measures.count_idle_cycles += 1

          # somatorio da duração de cada ciclo ocioso
          measures.sum_idle_cycles_time += birth - death

          if(VERBOSE > 1):
            print(f"servidor ocioso de {death:.3f} ate {birth:.3f}")
      system.arrival(current_event)

    # se o evento atual é de saída
    else:
      # somatório do número de clientes no sistema a cada momento
      measures.N += system.full_queue_size() * (T - last_T)

      # cliente termina de ser servido
      system.departure()

    serving = system.next(T)
    # se um cliente começou a ser servido neste momento
    if(serving):
      # somatório do tempo que cada cliente passa na fila de espera
      measures.W += T - serving.arrival_time

      # gerando o evento de saída do cliente do sistema
      serving.service_start = T
      serving.event_type = 'D'
      next_departure_event = serving

    # se o sistema está ocioso
    if(not(system.busy())):
      death = T
      # atualizamos várias métricas:

      # número de ciclos de vida e morte
      measures.count_birth_and_death_processes += 1

      # somatorio da duração de cada ciclo de vida e morte
      measures.sum_birth_and_death_processes_time += death - birth

      # numero total de clientes que passaram pelo sistema
      measures.TotalClients += count_clients

      if(VERBOSE > 1):
        print(f"servidor ficou ativo de {birth:.3f} ate {death:.3f} e atendeu {count_clients} clientes")
      count_clients = 0
      birth = -1

      if(VERBOSE > 1):
        print("")
    if(VERBOSE > 1):
      print(system)

    if(current_event.time > LIM_TIME): #condição de parada
      #measures.TotalClients += count_clients
      break


  # tempo total de simulação
  measures.sum_time = T

  if(VERBOSE > 0):
    print("Métricas obtidas:")
    print(f"E[T] =  {measures.Expected_Value_T():.3f}")
    print(f"E[W] =  {measures.Expected_Value_W():.3f}")
    print(f"E[X] =  {measures.Expected_Value_X():.3f}")
    print("")
    print(f"E[N] =  {measures.Expected_Value_N():.3f} --- Esperado: {(LAMBDA * measures.Expected_Value_X()):.3f}")
    print(f"E[Nq] =  {measures.Expected_Value_Nq():.3f}")
    print(f"E[Rho] =  {measures.Expected_Value_Rho():.3f}")

  return measures

simulation(1.05, 1, 100000, 1)

Métricas obtidas:
E[T] =  1111670.876
E[W] =  1111294.258
E[X] =  398.313

E[N] =  1435.006 --- Esperado: 418.228
E[Nq] =  1229960.255
E[Rho] =  0.002


<__main__.simulation.<locals>.Measures at 0x7fe0845e0910>

## Resultado das simulações

In [ ]:
def experiment(LAMBDA, MU, MAX_ITERATIONS = 5):
  LIM_TIME = 100000
  T = 1.96

  averages = {"T": 0.0, "W": 0.0, "X":0.0, "N":0.0, "Nq": 0.0, "Rho": 0.0}
  variances = {"T": 0.0, "W": 0.0, "X":0.0, "N":0.0, "Nq": 0.0, "Rho": 0.0}
  std_dev = {"T": 0.0, "W": 0.0, "X":0.0, "N":0.0, "Nq": 0.0, "Rho": 0.0}
  results = {"T": [], "W": [], "X":[], "N":[], "Nq":[], "Rho": []}
  confidence_interval = {"T": [], "W": [], "X":[], "N":[], "Nq":[], "Rho": []}


  def calc_average(s):
    n = len(s)
    total = 0
    for x in s:
      total += x
    return total / n

  def calc_variance(s, average):
    n = len(s)
    if(n == 1):
      return 0.0
    total = 0
    for x in s:
      total += (x - average) ** 2
    return total / (n - 1)


  def calc_confidence_interval(t):
    return [averages[key] - t * std_dev[key] / ((it+1)**0.5), averages[key] + t * std_dev[key] / ((it+1)**0.5)]

  for it in range(MAX_ITERATIONS):
    current = simulation(LAMBDA, MU, LIM_TIME)
    results["T"].append(current.Expected_Value_T())
    results["W"].append(current.Expected_Value_W())
    results["X"].append(current.Expected_Value_X())
    results["N"].append(current.Expected_Value_N())
    results["Nq"].append(current.Expected_Value_Nq())
    results["Rho"].append(current.Expected_Value_Rho())

    print(f"Simulacao {it+1}")
    for key, value in results.items():
      averages[key] = calc_average(value)
      variances[key] = calc_variance(value, averages[key])
      std_dev[key] = variances[key] ** (1/2)
      confidence_interval[key] = calc_confidence_interval(T)
      print(f" {key} média {averages[key]:.3f} variancia {variances[key]} desvio padrão {std_dev[key]} intervalo de confiança: ({confidence_interval[key][0]:.3f}, {confidence_interval[key][1]:.3f})")

    print()




Abaixo apresentamos o resultado dos experimentos, cada um com 5 simulações com os valores sugeridos no enunciado e LIM_TIME = 100000.

In [ ]:
experiment(1, 2)

Simulacao 1
 T média 1.006 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.006, 1.006)
 W média 0.503 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.503, 0.503)
 X média 0.503 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.503, 0.503)
 N média 0.667 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.667, 0.667)
 Nq média 0.501 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.501, 0.501)
 Rho média 0.499 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.499, 0.499)

Simulacao 2
 T média 0.999 variancia 9.578441557259613e-05 desvio padrão 0.009786951291009684 intervalo de confiança: (0.985, 1.012)
 W média 0.497 variancia 7.049819985546002e-05 desvio padrão 0.008396320614141651 intervalo de confiança: (0.486, 0.509)
 X média 0.502 variancia 1.9338536794534337e-06 desvio padrão 0.0013906306768705463 intervalo de confiança: (0.500, 0.504)
 N média 0.665 variancia 9.311341452821976e-06 desvio padrão 0.0030514490742632387 interval

In [ ]:
experiment(2, 4)

Simulacao 1
 T média 0.499 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.499, 0.499)
 W média 0.249 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.249, 0.249)
 X média 0.250 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.250, 0.250)
 N média 0.665 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.665, 0.665)
 Nq média 0.499 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.499, 0.499)
 Rho média 0.499 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.499, 0.499)

Simulacao 2
 T média 0.500 variancia 2.2034812599057654e-06 desvio padrão 0.0014844127660141452 intervalo de confiança: (0.498, 0.502)
 W média 0.250 variancia 2.535643768427453e-06 desvio padrão 0.001592370487175473 intervalo de confiança: (0.248, 0.252)
 X média 0.250 variancia 1.1654869558050345e-08 desvio padrão 0.00010795772115995384 intervalo de confiança: (0.250, 0.250)
 N média 0.668 variancia 1.6908503198508518e-05 desvio padrão 0.004111995038726155 inter

In [ ]:
experiment(1.05, 1)

Simulacao 1
 T média 2954.245 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (2954.245, 2954.245)
 W média 2953.238 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (2953.238, 2953.238)
 X média 1.008 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.008, 1.008)
 N média 1504.089 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1504.089, 1504.089)
 Nq média 2928.030 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (2928.030, 2928.030)
 Rho média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)

Simulacao 2
 T média 2665.291 variancia 166989.3106373076 desvio padrão 408.6432559547601 intervalo de confiança: (2098.940, 3231.642)
 W média 2664.288 variancia 166983.92237557395 desvio padrão 408.63666303401357 intervalo de confiança: (2097.946, 3230.630)
 X média 1.003 variancia 4.346660409637239e-05 desvio padrão 0.006592920756111997 intervalo de confiança: (0.994, 1.012)
 N média 1359.148 variancia 42015.3165594885 desvio

In [ ]:
experiment(1.10, 1)

Simulacao 1
 T média 4939.671 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (4939.671, 4939.671)
 W média 4938.670 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (4938.670, 4938.670)
 X média 1.001 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.001, 1.001)
 N média 2587.851 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (2587.851, 2587.851)
 Nq média 4931.242 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (4931.242, 4931.242)
 Rho média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)

Simulacao 2
 T média 5005.944 variancia 8784.02768662095 desvio padrão 93.72314381528689 intervalo de confiança: (4876.050, 5135.837)
 W média 5004.944 variancia 8784.260409801187 desvio padrão 93.72438535301892 intervalo de confiança: (4875.048, 5134.839)
 X média 1.000 variancia 1.5414159663052153e-06 desvio padrão 0.001241537742601978 intervalo de confiança: (0.998, 1.002)
 N média 2615.582 variancia 1537.948561068317 desvio 

Abaixo temos uma célula em que podemos testar diferentes valores de $\mu$, $\lambda$ e LIM_TIME.

Além disso, há um parâmetro extra booleano VERBOSE.

Quando passamos VERBOSE = 2 nós mostramos cada evento de chegada e saída de clientes do sistema, além dos resultados das métricas calculadas.

Quando passamos VERBOSE = 1 nós mostramos somente resultados das métricas calculadas.


Quando passamos VERBOSE = 0 não mostramos nada.


In [ ]:
print("   SIMULAÇÃO   ")
LAMBDA = float(input("Taxa lambda (momentos de chegada) : "))
MU = float(input("Taxa mu (vida residual) : "))
LIM_TIME = float(input("Tempo máximo aceitando novos clientes (s): "))
VERBOSE = int(input("Verbose? (0 - Não imprime nada; 2 - Imprime métricas obtidas; 3 - Imprime a simulação dos eventos e métricas obtidas)"))
simulation(LAMBDA, MU, LIM_TIME, VERBOSE)

   SIMULAÇÃO   
Taxa lambda (momentos de chegada) : 1.1
Taxa mu (vida residual) : 1
Tempo máximo aceitando novos clientes (s): 23
Verbose? (0 - Não imprime nada; 2 - Imprime métricas obtidas; 3 - Imprime a simulação dos eventos e métricas obtidas)3
Arrival event: 
id	|	chegada	|	servidor|	saída	|

0	|	0.000	|

Current event time: 0.000

	Servidor: 
	id	|	chegada	|	servidor|	saída	|
	0	|	0.000	|	

 	Fila de Espera: (0) 
	id	|	chegada	|
---------------------------------------

Departure event: 
id	|	chegada	|	servidor|	saída	|

0	|	0.000	|	

Current event time: 1.424

servidor ficou ativo de 0.000 ate 1.424 e atendeu 1 clientes

	Servidor: 
	id	|	chegada	|	servidor|	saída	|
	None

 	Fila de Espera: (0) 
	id	|	chegada	|
---------------------------------------

Arrival event: 
id	|	chegada	|	servidor|	saída	|

1	|	3.011	|

Current event time: 3.011

servidor ocioso de 1.424 ate 3.011
	Servidor: 
	id	|	chegada	|	servidor|	saída	|
	1	|	3.011	|	

 	Fila de Espera: (0) 
	id	|	chegada	|
-------